# Проверка кода агента

Используйте ноутбук, чтобы по шагам запускать загрузку моков, выбор фреймворка и кодогенерацию.


In [4]:
from mas_automl.code_agent.load_mocks import load_mock_inputs

DATA_ANALYZE, METADATA, REGISTRY = load_mock_inputs()
print(f"Ключи data: {list[str](DATA_ANALYZE.keys())}")
print(f"Ключи metadata: {list(METADATA.keys())[:5]}")
print(f"Фреймворки: {list(REGISTRY.keys())}")


Ключи data: ['summary', 'priority', 'steps', 'example_pipeline_snippet', 'frameworks_recommended', 'rationale', 'estimated_complexity', 'confidence']
Ключи metadata: ['dataset_id', 'name', 'version', 'version_label', 'description']
Фреймворки: ['AutoGluon', 'H2O AutoML', 'LightAutoML']


In [5]:
METADATA

{'dataset_id': 31,
 'name': 'credit-g',
 'version': 1,
 'version_label': '1',
 'description': '**Author**: Dr. Hans Hofmann  \n**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) - 1994    \n**Please cite**: [UCI](https://archive.ics.uci.edu/ml/citation_policy.html)\n\n**German Credit dataset**  \nThis dataset classifies people described by a set of attributes as good or bad credit risks.\n\nThis dataset comes with a cost matrix: \n``` \nGood  Bad (predicted)  \nGood   0    1   (actual)  \nBad    5    0  \n```\n\nIt is worse to class a customer as good when they are bad (5), than it is to class a customer as bad when they are good (1).  \n\n### Attribute description  \n\n1. Status of existing checking account, in Deutsche Mark.  \n2. Duration in months  \n3. Credit history (credits taken, paid back duly, delays, critical accounts)  \n4. Purpose of the credit (car, television,...)  \n5. Credit amount  \n6. Status of savings account/bonds, in Deutsche 

In [6]:
# DATA_ANALYZE, METADATA, REGISTRY
import pandas as pd

PATH_TO_CSV = METADATA.get('local_path')
dataset_df = pd.read_csv(PATH_TO_CSV)

dataset_df.info(), dataset_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   checking_status         1000 non-null   object 
 1   duration                1000 non-null   int64  
 2   credit_history          1000 non-null   object 
 3   purpose                 1000 non-null   object 
 4   credit_amount           1000 non-null   float64
 5   savings_status          1000 non-null   object 
 6   employment              1000 non-null   object 
 7   installment_commitment  1000 non-null   int64  
 8   personal_status         1000 non-null   object 
 9   other_parties           1000 non-null   object 
 10  residence_since         1000 non-null   int64  
 11  property_magnitude      1000 non-null   object 
 12  age                     1000 non-null   int64  
 13  other_payment_plans     1000 non-null   object 
 14  housing                 1000 non-null   o

(None,
   checking_status  duration                  credit_history   purpose  \
 0              <0         6  critical/other existing credit  radio/tv   
 1        0<=X<200        48                   existing paid  radio/tv   
 
    credit_amount    savings_status employment  installment_commitment  \
 0         1169.0  no known savings        >=7                       4   
 1         5951.0              <100     1<=X<4                       2   
 
       personal_status other_parties  ...  property_magnitude age  \
 0         male single          none  ...         real estate  67   
 1  female div/dep/mar          none  ...         real estate  22   
 
    other_payment_plans housing existing_credits      job num_dependents  \
 0                 none     own                2  skilled              1   
 1                 none     own                1  skilled              1   
 
    own_telephone foreign_worker class  
 0            yes            yes  good  
 1           none       

In [1]:
# --- Инициализация клиента ---
from mas_automl.code_agent.openai_wraper import LLMClient, LLMConfig

client = LLMClient(LLMConfig())

# --- Вызов выбора фреймворка ---


🔑 OPENAI_API_KEY = sk-or-v
🌐 BASE_URL = https://openrouter.ai/api/v1
KWARGS - {'model': 'qwen/qwen3-coder-30b-a3b-instruct', 'temperature': 0.0, 'openai_api_key': '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', 'openai_api_base': 'https://openrouter.ai/api/v1'}


C:\Users\User1\Desktop\ITMO_bootcamp\src\mas_automl\code_agent\openai_wraper.py:50: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import ChatOpenAI``.
  self._client = ChatOpenAI(**client_kwargs)


✅ ChatOpenAI клиент инициализирован для OpenRouter


In [2]:
client.chat('loh')

'Hello! It looks like you sent "loh" - is there something I can help you with? I\'m here to assist with questions, conversations, or any other support you might need. Feel free to let me know how I can help! 😊'

## Кодогенерация

In [ ]:
from mas_automl.code_agent.base_pipeline import choose_framework

framework, reason = choose_framework(DATA_ANALYZE, METADATA, REGISTRY, client)

# --- Вывод результата ---

print("🔧 Выбранный фреймворк:", framework)
print("📘 Обоснование:", reason)

In [4]:
from mas_automl.code_agent.base_pipeline import generate_code, evaluate_code

code_sample = generate_code(framework, client, iteration=1, feedback="")
print(code_sample)
print(evaluate_code(code_sample, framework))


import pandas as pd
from autogluon.tabular import TabularPredictor

def train_model(train_df: pd.DataFrame, test_df: pd.DataFrame, label: str) -> TabularPredictor:
    predictor = TabularPredictor(label=label)
    predictor.fit(train_data=train_df, presets='medium_quality', time_limit=60)
    predictor.evaluate(test_df, silent=True)
    return predictor
(True, 'Проверки пройдены.')


In [5]:
from mas_automl.code_agent.base_pipeline import run_pipeline

result = run_pipeline(max_iterations=2, llm=client)
result


PipelineResult(framework='AutoGluon', reason='Эвристика на основе типа задачи и размера выборки.', code="import pandas as pd\nfrom autogluon.tabular import TabularPredictor\n\ndef train_model(train_df: pd.DataFrame, test_df: pd.DataFrame, label: str) -> TabularPredictor:\n    predictor = TabularPredictor(label=label)\n    predictor.fit(train_data=train_df, presets='medium_quality', time_limit=60)\n    predictor.evaluate(test_df, silent=True)\n    return predictor", tests_passed=True, iterations=1, feedback='Проверки пройдены.')